In [24]:
import requests
from datetime import datetime, timedelta
import calendar
import json
import time

# VERIFICACIÓN DE CONEXIÓN CON LA API DE BUK
# =====================================================================

# Reemplaza esta URL con el enlace de tu API
url_api = "https://demo.buk.cl/apidocs#/"

# 2. AUTENTICACIÓN CON API KEY EN LA CABECERA
headers = {
    "X-API-Key": "JVZhehjdRWdmx2reD5mkKRHA"
}
params = {}

print(f"Comprobando la conexión con la API en: {url_api}")

try:
    # Envía la solicitud con las cabeceras y/o parámetros de autenticación
    response = requests.get(url_api, headers=headers, params=params, timeout=5)

    # El método raise_for_status() lanza una excepción si la respuesta no es 2xx
    response.raise_for_status()

    print("---")
    print("✅ ¡Conexión y autenticación exitosas!")
    print(f"Código de estado de la respuesta: {response.status_code}")

except requests.exceptions.HTTPError as err_http:
    print("---")
    print(f"❌ Error HTTP: {err_http}")
    print(f"El servidor respondió con el código: {response.status_code}")
    print("Verifica si las credenciales de autenticación son correctas.")

except requests.exceptions.ConnectionError as err_conn:
    print("---")
    print(f"❌ Error de conexión: {err_conn}")
    print("Verifica tu conexión a internet o la URL de la API.")

except requests.exceptions.Timeout as err_timeout:
    print("---")
    print(f"❌ Error de tiempo de espera: {err_timeout}")

except requests.exceptions.RequestException as err:
    print("---")
    print(f"❌ Ocurrió un error inesperado: {err}")
    


Comprobando la conexión con la API en: https://demo.buk.cl/apidocs#/
---
✅ ¡Conexión y autenticación exitosas!
Código de estado de la respuesta: 200


In [ ]:
# EXTRACCIÓN MASIVA DE DATOS BUK - INASISTENCIAS
# =====================================================================

# Usar la URL correcta de BUK (mismo dominio que el primer bloque)
url_inasistencias = "https://demo.buk.cl/api/v1/chile/absences/absence"

# Verificar que los headers del bloque 1 estén disponibles
if 'headers' not in globals():
    print("❌ Headers no definidos - ejecuta el bloque 1 primero")
    # Crear headers de respaldo usando la misma configuración del bloque 1
    headers = {
        "X-API-Key": "JVZhehjdRWdmx2reD5mkKRHA"
    }

print(f"🧪 PRUEBA INICIAL DE CONEXIÓN:")
print(f"🔗 URL: {url_inasistencias}")
print(f"🔑 Headers: {headers}")

# Prueba inicial rápida
response_test = requests.get(url_inasistencias, headers=headers, timeout=10)
print(f"📊 Status de prueba: {response_test.status_code}")
print("-" * 60)

# Rango de fechas
global_start_date = datetime(2022, 1, 1) 
global_end_date = datetime.now() 

all_data = []
current_date = global_start_date
request_count = 0
error_count = 0

print(f"🚀 INICIANDO EXTRACCIÓN MASIVA DE INASISTENCIAS BUK")
print(f"📅 Período: {global_start_date.strftime('%Y-%m-%d')} hasta {global_end_date.strftime('%Y-%m-%d')}")
print(f"🔗 URL base: {url_inasistencias}")
print(f"🔑 Headers validados: {headers}")

# Solo continuar si la prueba inicial fue exitosa
if response_test.status_code not in [200, 204]:
    print(f"❌ PROBLEMA DE CONEXIÓN INICIAL - Status: {response_test.status_code}")
    print(f"📄 Respuesta: {response_test.text[:200]}")
    print("⏸️ No se ejecutará la extracción masiva")
else:
    print("✅ Conexión inicial exitosa - procediendo con extracción")

print("-" * 60)

# 2. Bucle para iterar por cada mes dentro del rango global
while current_date < global_end_date:
    # 3. Definir el sub-rango (mes actual)
    from_date = current_date
    last_day_of_month = calendar.monthrange(current_date.year, current_date.month)[1]
    to_date = datetime(current_date.year, current_date.month, last_day_of_month, 23, 59, 59)
    
    # 4. Asegurarse de que el 'to_date' no exceda el 'global_end_date'
    if to_date > global_end_date:
        to_date = global_end_date

    # 5. Formatear las fechas para la solicitud de la API
    from_date_str = from_date.isoformat()
    to_date_str = to_date.isoformat()

    # 6. Construir la URL completa y parámetros
    full_url = url_inasistencias  # URL ya completa
    
    params = {
        'from': from_date_str,
        'to': to_date_str
    }
    
    print(f"📅 Procesando: {from_date.strftime('%Y-%m-%d')} a {to_date.strftime('%Y-%m-%d')}")
    
    # Solo continuar si la prueba inicial fue exitosa
    if response_test.status_code not in [200, 204]:
        print("   ⏸️ Saltando - problema de conexión inicial")
        break
    
    try:
        # Realizar la solicitud GET con timeout y headers
        response = requests.get(
            full_url, 
            params=params, 
            headers=headers,
            timeout=30  # 30 segundos de timeout
        )
        
        request_count += 1
        
        # Verificar el código de estado
        print(f"   📊 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("   ✅ Respuesta exitosa")
            
            # Verificar si hay contenido en la respuesta
            if not response.content:
                print("   📭 Respuesta vacía - saltando...")
                current_date = to_date + timedelta(days=1)
                continue
            
            # Verificar el tipo de contenido
            content_type = response.headers.get('content-type', '')
            print(f"   📄 Tipo de contenido: {content_type}")
            
            if 'application/json' not in content_type.lower():
                print(f"   ⚠️ Tipo de contenido inesperado")
                print(f"   📄 Contenido de respuesta: {response.text[:200]}...")
                current_date = to_date + timedelta(days=1)
                continue
            
            # Intentar parsear JSON
            try:
                data = response.json()
                print(f"   ✅ JSON parseado correctamente - Tipo: {type(data)}")
                
                # Procesar según la estructura de respuesta de BUK
                results = []
                
                if isinstance(data, list):
                    # Lista directa de inasistencias
                    results = data
                    print(f"   📊 Lista directa: {len(results)} registros")
                    
                elif isinstance(data, dict):
                    # Buscar en estructuras wrapper comunes
                    possible_keys = ['data', 'results', 'items', 'records', 'inasistencias', 'absences']
                    
                    for key in possible_keys:
                        if key in data and isinstance(data[key], list):
                            results = data[key]
                            print(f"   📊 Datos en '{key}': {len(results)} registros")
                            break
                    
                    if not results and data:
                        # Si no encuentra estructura conocida, mostrar claves disponibles
                        print(f"   📋 Claves en respuesta: {list(data.keys())}")
                        # Usar toda la respuesta como un registro si tiene datos válidos
                        if any(key in data for key in ['id', 'employee_id', 'start_date']):
                            results = [data]
                            print(f"   📄 Usando respuesta completa como registro único")
                        else:
                            print(f"   📄 Muestra de respuesta: {str(data)[:300]}...")
                            
                else:
                    print(f"   ⚠️ Estructura inesperada: {type(data)}")
                
                # Validar y procesar registros
                if results:
                    valid_results = []
                    
                    for item in results:
                        if isinstance(item, dict):
                            # Verificar campos mínimos esperados
                            required_fields = ['id']  # Campo mínimo requerido
                            if any(field in item for field in required_fields):
                                valid_results.append(item)
                            else:
                                print(f"   ⚠️ Registro sin campos requeridos: {list(item.keys())[:5]}")
                        else:
                            print(f"   ⚠️ Elemento no es diccionario: {str(item)[:50]}...")
                    
                    if valid_results:
                        all_data.extend(valid_results)
                        print(f"   📈 Registros válidos agregados: {len(valid_results)}")
                        
                        # Mostrar muestra del primer registro válido
                        if len(valid_results) > 0:
                            sample = valid_results[0]
                            print(f"   🔍 Campos del primer registro: {list(sample.keys())[:8]}{'...' if len(sample.keys()) > 8 else ''}")
                        
                        if len(valid_results) != len(results):
                            filtered = len(results) - len(valid_results)
                            print(f"   ⚠️ Se filtraron {filtered} registros inválidos")
                    else:
                        print(f"   📭 No se encontraron registros válidos")
                        
                else:
                    print(f"   📭 No hay datos en este rango")
                    
            except json.JSONDecodeError as json_error:
                error_count += 1
                print(f"   ❌ Error JSON: {json_error}")
                print(f"   📄 Respuesta recibida: {response.text[:500]}...")
                
                # Decidir si continuar según el error
                if response.status_code in [404, 204]:  # No encontrado o sin contenido
                    print("   🔄 Continuando con el siguiente rango...")
                else:
                    print("   ⏸️ Pausando 5 segundos antes de continuar...")
                    time.sleep(5)
                
        elif response.status_code == 401:
            print("   ❌ Error de autenticación (401). Verifica tu token.")
            break
        elif response.status_code == 403:
            print("   ❌ Acceso denegado (403). Verifica permisos.")
            break
        elif response.status_code == 404:
            print("   ⚠️ Endpoint no encontrado (404) - continuando...")
        elif response.status_code == 204:
            print("   📭 Sin contenido (204) - rango sin datos - continuando...")
        elif response.status_code == 429:
            print("   ⏳ Rate limit alcanzado (429). Esperando 60 segundos...")
            time.sleep(60)
            continue  # Reintentar la misma fecha
        elif response.status_code == 500:
            print("   ⚠️ Error del servidor (500) - pausando y continuando...")
            time.sleep(10)
        else:
            print(f"   ⚠️ Código inesperado: {response.status_code}")
            print(f"   📄 Respuesta: {response.text[:200]}...")
            error_count += 1
            
    except requests.exceptions.Timeout:
        error_count += 1
        print(f"   ⏰ Timeout en la solicitud")
    except requests.exceptions.ConnectionError:
        error_count += 1
        print(f"   🌐 Error de conexión")
    except requests.exceptions.RequestException as e:
        error_count += 1
        print(f"   ❌ Error en la solicitud: {e}")
        
    except Exception as e:
        error_count += 1
        print(f"   💥 Error inesperado: {e}")
    
    # 8. Pausa entre requests y avanzar al siguiente período
    current_date = to_date + timedelta(days=1)
    time.sleep(1)  # Pausa para evitar rate limiting
    
    # Mostrar progreso cada ciertos registros
    if request_count % 10 == 0 and request_count > 0:
        print(f"\n📊 PROGRESO: {request_count} requests | {len(all_data)} registros | {error_count} errores")

print("\n" + "="*70)
print(f"🏁 EXTRACCIÓN FINALIZADA")
print(f"📊 Registros extraídos: {len(all_data)}")
print(f"📡 Requests realizados: {request_count}")
print(f"❌ Errores encontrados: {error_count}")
print(f"⏱️ Período procesado: {global_start_date.strftime('%Y-%m-%d')} a {global_end_date.strftime('%Y-%m-%d')}")

if all_data:
    print(f"\n🔍 MUESTRA DE DATOS EXTRAÍDOS:")
    sample = all_data[0]
    print(f"   📋 Total de campos: {len(sample.keys()) if isinstance(sample, dict) else 'N/A'}")
    
    if isinstance(sample, dict):
        # Mostrar campos más importantes primero
        priority_fields = ['id', 'employee_id', 'start_date', 'end_date', 'type', 'status']
        shown_fields = []
        
        for field in priority_fields:
            if field in sample:
                print(f"   - {field}: {sample[field]}")
                shown_fields.append(field)
        
        # Mostrar otros campos si hay espacio
        other_fields = [k for k in list(sample.keys())[:8] if k not in shown_fields][:3]
        for field in other_fields:
            print(f"   - {field}: {sample[field]}")
            
    print(f"\n✅ Datos listos para análisis - ejecuta el siguiente bloque")
    
else:
    print(f"\n❌ NO SE OBTUVIERON DATOS")
    print(f"💡 Posibles causas:")
    print(f"   1. Rango de fechas sin datos")
    print(f"   2. Configuración incorrecta de headers")
    print(f"   3. Endpoint o parámetros incorrectos")
    print(f"   4. Problemas de conectividad")
    print(f"\n🔧 Sugerencias:")
    print(f"   1. Verifica la conexión ejecutando el bloque 1")
    print(f"   2. Prueba con un rango de fechas más reciente")
    print(f"   3. Revisa los errores mostrados arriba")

Respuesta de Inasistencias: 401
🚀 INICIANDO EXTRACCIÓN MASIVA DE INASISTENCIAS BUK
📅 Período: 2022-01-01 hasta 2025-08-22
🔗 URL base: https://cramer.buk.cl/api/v1/chile/absences/absence
🔑 Headers: {'X-API-Key': 'JVZhehjdRWdmx2reD5mkKRHA'}
------------------------------------------------------------
📅 Procesando: 2022-01-01 a 2022-01-31
   📊 Status Code: 401
   ❌ Error de autenticación (401). Verifica tu token.

🏁 EXTRACCIÓN FINALIZADA
📊 Registros extraídos: 0
📡 Requests realizados: 1
❌ Errores encontrados: 0
⏱️ Período procesado: 2022-01-01 a 2025-08-22

❌ NO SE OBTUVIERON DATOS
💡 Posibles causas:
   1. Rango de fechas sin datos
   2. Configuración incorrecta de headers
   3. Endpoint o parámetros incorrectos
   4. Problemas de conectividad

🔧 Sugerencias:
   1. Verifica la conexión ejecutando el bloque 1
   2. Prueba con un rango de fechas más reciente
   3. Revisa los errores mostrados arriba


In [9]:
# CONVERTIR DATOS A DATAFRAME Y GUARDAR
# =====================================================================

import pandas as pd
import os

if all_data:
    print("📊 Convirtiendo datos a DataFrame...")
    
    # Crear DataFrame
    df_inasistencias = pd.DataFrame(all_data)
    
    print(f"✅ DataFrame creado: {df_inasistencias.shape[0]} filas × {df_inasistencias.shape[1]} columnas")
    
    # Mostrar información del DataFrame
    print("\n📋 INFORMACIÓN DEL DATAFRAME:")
    print(f"Columnas disponibles: {list(df_inasistencias.columns)}")
    print(f"Tipos de datos:\n{df_inasistencias.dtypes}")
    
    # Mostrar las primeras filas
    print("\n📄 PRIMERAS 5 FILAS:")
    print(df_inasistencias.head())
    
    # Crear carpeta para guardar si no existe
    carpeta_datos = "datos_buk"
    os.makedirs(carpeta_datos, exist_ok=True)
    
    # Guardar en diferentes formatos con timestamp
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Excel
    excel_path = f"{carpeta_datos}/inasistencias_buk_{timestamp}.xlsx"
    df_inasistencias.to_excel(excel_path, index=False, engine='openpyxl')
    print(f"\n💾 Guardado en Excel: {excel_path}")
    
    # CSV
    csv_path = f"{carpeta_datos}/inasistencias_buk_{timestamp}.csv"
    df_inasistencias.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"💾 Guardado en CSV: {csv_path}")
    
    # JSON (backup)
    json_path = f"{carpeta_datos}/inasistencias_buk_{timestamp}.json"
    df_inasistencias.to_json(json_path, orient='records', indent=2, force_ascii=False)
    print(f"💾 Guardado en JSON: {json_path}")
    
    # Estadísticas básicas
    print(f"\n📈 ESTADÍSTICAS:")
    print(f"   - Registros totales: {len(df_inasistencias)}")
    print(f"   - Período: {global_start_date.strftime('%Y-%m-%d')} a {global_end_date.strftime('%Y-%m-%d')}")
    print(f"   - Requests realizados: {request_count}")
    print(f"   - Errores encontrados: {error_count}")
    
    # Si hay columnas de fecha, mostrar rangos
    date_columns = [col for col in df_inasistencias.columns if 'date' in col.lower() or 'fecha' in col.lower()]
    if date_columns:
        print(f"\n📅 RANGOS DE FECHAS EN LOS DATOS:")
        for col in date_columns:
            try:
                df_inasistencias[col] = pd.to_datetime(df_inasistencias[col])
                min_date = df_inasistencias[col].min()
                max_date = df_inasistencias[col].max()
                print(f"   - {col}: {min_date} a {max_date}")
            except:
                print(f"   - {col}: No se pudo convertir a fecha")

else:
    print("❌ No hay datos para convertir a DataFrame")
    print("🔧 Sugerencias para solucionar:")
    print("   1. Verifica que la URL esté correcta")
    print("   2. Asegúrate de tener un token válido")
    print("   3. Revisa los logs de error arriba para más detalles")
    print("   4. Prueba con un rango de fechas más pequeño")

❌ No hay datos para convertir a DataFrame
🔧 Sugerencias para solucionar:
   1. Verifica que la URL esté correcta
   2. Asegúrate de tener un token válido
   3. Revisa los logs de error arriba para más detalles
   4. Prueba con un rango de fechas más pequeño


In [10]:
# ANÁLISIS ESPECÍFICO DE DATOS BUK - INASISTENCIAS
# =====================================================================

if all_data and len(all_data) > 0:
    print("\n" + "="*70)
    print("📊 ANÁLISIS ESPECÍFICO DE INASISTENCIAS BUK")
    print("="*70)
    
    # Análisis por campos específicos de BUK
    print(f"\n🔍 ANÁLISIS POR CAMPOS CLAVE:")
    
    # 1. Análisis por tipo de inasistencia
    if 'type' in df_inasistencias.columns:
        tipos = df_inasistencias['type'].value_counts()
        print(f"\n📋 TIPOS DE INASISTENCIAS:")
        for tipo, cantidad in tipos.items():
            print(f"   - {tipo}: {cantidad} casos")
    
    # 2. Análisis por estado
    if 'status' in df_inasistencias.columns:
        estados = df_inasistencias['status'].value_counts()
        print(f"\n✅ ESTADOS DE SOLICITUDES:")
        for estado, cantidad in estados.items():
            print(f"   - {estado}: {cantidad} casos")
    
    # 3. Análisis por días de duración
    if 'days_count' in df_inasistencias.columns:
        df_inasistencias['days_count'] = pd.to_numeric(df_inasistencias['days_count'], errors='coerce')
        print(f"\n📅 ESTADÍSTICAS DE DURACIÓN:")
        print(f"   - Promedio de días: {df_inasistencias['days_count'].mean():.2f}")
        print(f"   - Mediana: {df_inasistencias['days_count'].median():.1f}")
        print(f"   - Máximo: {df_inasistencias['days_count'].max():.0f} días")
        print(f"   - Mínimo: {df_inasistencias['days_count'].min():.0f} días")
    
    # 4. Análisis temporal
    if 'start_date' in df_inasistencias.columns:
        df_inasistencias['start_date'] = pd.to_datetime(df_inasistencias['start_date'])
        
        # Análisis por año
        df_inasistencias['año'] = df_inasistencias['start_date'].dt.year
        años = df_inasistencias['año'].value_counts().sort_index()
        print(f"\n📆 DISTRIBUCIÓN POR AÑOS:")
        for año, cantidad in años.items():
            print(f"   - {año}: {cantidad} inasistencias")
        
        # Análisis por mes
        df_inasistencias['mes'] = df_inasistencias['start_date'].dt.month
        meses = df_inasistencias['mes'].value_counts().sort_index()
        print(f"\n📅 DISTRIBUCIÓN POR MESES:")
        nombres_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                        'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
        for mes, cantidad in meses.items():
            print(f"   - {nombres_meses[mes-1]}: {cantidad} casos")
    
    # 5. Análisis por empleados
    if 'employee_id' in df_inasistencias.columns:
        empleados_unicos = df_inasistencias['employee_id'].nunique()
        empleados_mas_inasistencias = df_inasistencias['employee_id'].value_counts().head(10)
        print(f"\n👥 ANÁLISIS DE EMPLEADOS:")
        print(f"   - Total empleados con inasistencias: {empleados_unicos}")
        print(f"   - Promedio inasistencias por empleado: {len(df_inasistencias)/empleados_unicos:.2f}")
        print(f"\n   🔝 TOP 5 EMPLEADOS CON MÁS INASISTENCIAS:")
        for empleado_id, cantidad in empleados_mas_inasistencias.head(5).items():
            print(f"      - Empleado ID {empleado_id}: {cantidad} inasistencias")
    
    # 6. Análisis por tipo de licencia
    if 'licence_type' in df_inasistencias.columns:
        tipos_licencia = df_inasistencias['licence_type'].value_counts()
        print(f"\n📜 TIPOS DE LICENCIAS:")
        for tipo, cantidad in tipos_licencia.items():
            print(f"   - {tipo}: {cantidad} casos")
    
    # 7. Análisis de formatos
    if 'format' in df_inasistencias.columns:
        formatos = df_inasistencias['format'].value_counts()
        print(f"\n💻 FORMATOS DE LICENCIAS:")
        for formato, cantidad in formatos.items():
            print(f"   - {formato}: {cantidad} casos")
    
    # 8. Resumen de calidad de datos
    print(f"\n🔍 CALIDAD DE DATOS:")
    campos_importantes = ['id', 'employee_id', 'start_date', 'end_date', 'type', 'status']
    for campo in campos_importantes:
        if campo in df_inasistencias.columns:
            nulos = df_inasistencias[campo].isnull().sum()
            porcentaje = (nulos / len(df_inasistencias)) * 100
            print(f"   - {campo}: {nulos} nulos ({porcentaje:.1f}%)")
    
    # Guardar análisis en archivo separado
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    analisis_path = f"datos_buk/analisis_inasistencias_{timestamp}.txt"
    
    with open(analisis_path, 'w', encoding='utf-8') as f:
        f.write("ANÁLISIS DE INASISTENCIAS BUK\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Período analizado: {global_start_date.strftime('%Y-%m-%d')} a {global_end_date.strftime('%Y-%m-%d')}\n")
        f.write(f"Total de registros: {len(df_inasistencias)}\n")
        f.write(f"Empleados únicos: {df_inasistencias['employee_id'].nunique() if 'employee_id' in df_inasistencias.columns else 'N/A'}\n")
        f.write(f"Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        if 'type' in df_inasistencias.columns:
            f.write("TIPOS DE INASISTENCIAS:\n")
            tipos = df_inasistencias['type'].value_counts()
            for tipo, cantidad in tipos.items():
                f.write(f"  - {tipo}: {cantidad}\n")
    
    print(f"\n💾 Análisis detallado guardado en: {analisis_path}")

else:
    print("❌ No hay datos de inasistencias para analizar")

❌ No hay datos de inasistencias para analizar


In [ ]:
# FUNCIÓN DE PRUEBA PARA DEBUGGING
# =====================================================================

def probar_api_buk_simple():
    """
    Función para probar la API BUK con un rango pequeño y ver la respuesta exacta
    """
    print("🧪 PRUEBA SIMPLE DE API BUK")
    print("=" * 40)
    
    # Configuración de prueba (usar la misma URL del bloque principal)
    test_url = "https://demo.buk.cl/api/v1/chile/absences/absence"
    test_headers = {
        "X-API-Key": "JVZhehjdRWdmx2reD5mkKRHA"  # Mismo formato que el bloque 1
    }
    
    # Prueba con un rango pequeño reciente
    test_params = {
        'from': '2024-01-01T00:00:00',
        'to': '2024-01-31T23:59:59'
    }
    
    print(f"🔗 URL: {test_url}")
    print(f"📋 Parámetros: {test_params}")
    
    try:
        response = requests.get(test_url, headers=test_headers, params=test_params, timeout=30)
        
        print(f"\n📊 RESPUESTA:")
        print(f"   Status Code: {response.status_code}")
        print(f"   Headers: {dict(response.headers)}")
        print(f"   Tamaño respuesta: {len(response.content)} bytes")
        
        if response.status_code == 200:
            if response.content:
                try:
                    data = response.json()
                    print(f"   Tipo de datos: {type(data)}")
                    
                    if isinstance(data, list):
                        print(f"   📋 Lista con {len(data)} elementos")
                        if data:
                            print(f"   🔍 Primer elemento: {data[0]}")
                            print(f"   📚 Claves disponibles: {list(data[0].keys()) if isinstance(data[0], dict) else 'N/A'}")
                    elif isinstance(data, dict):
                        print(f"   📖 Diccionario con claves: {list(data.keys())}")
                        print(f"   📄 Contenido: {str(data)[:500]}...")
                    
                    return data
                    
                except json.JSONDecodeError as e:
                    print(f"   ❌ Error JSON: {e}")
                    print(f"   📄 Contenido raw: {response.text[:500]}...")
            else:
                print("   ⚠️  Respuesta vacía")
        else:
            print(f"   ❌ Error HTTP: {response.status_code}")
            print(f"   📄 Mensaje: {response.text[:300]}...")
            
    except Exception as e:
        print(f"   💥 Error: {e}")
    
    return None

# Ejecutar prueba simple
print("🔧 Ejecuta 'probar_api_buk_simple()' para probar la API con un rango pequeño")

# Función para verificar diferentes endpoints posibles
def probar_endpoints_buk():
    """
    Prueba diferentes endpoints posibles para inasistencias
    """
    print("🔍 PROBANDO DIFERENTES ENDPOINTS BUK")
    print("=" * 45)
    
    base_urls = [
        "https://demo.buk.cl/api/v1/chile",  # URL principal del bloque 1
        "https://cramer.buk.cl/api/v1/chile",
        "https://cramer.buk.cl/api/v1", 
        "https://demo.buk.cl/api/v1",
        "https://api.buk.cl/v1"
    ]
    
    endpoints = [
        "absences/absence",
        "absences", 
        "inasistencias",
        "Inasistencias", 
        "leaves",
        "attendances"
    ]
    
    headers = {
        "X-API-Key": "JVZhehjdRWdmx2reD5mkKRHA"  # Mismo formato que el bloque 1
    }
    
    for base in base_urls:
        for endpoint in endpoints:
            test_url = f"{base}/{endpoint}"
            print(f"\n🧪 Probando: {test_url}")
            
            try:
                response = requests.get(test_url, headers=headers, timeout=10)
                print(f"   Status: {response.status_code}")
                
                if response.status_code == 200:
                    print(f"   ✅ ¡ENDPOINT VÁLIDO ENCONTRADO!")
                    if response.content:
                        try:
                            data = response.json()
                            print(f"   📋 Tipo: {type(data)}, Tamaño: {len(data) if isinstance(data, list) else 'dict'}")
                            return test_url
                        except:
                            print(f"   📄 Respuesta no-JSON")
                elif response.status_code == 404:
                    print(f"   ❌ No encontrado")
                elif response.status_code == 401:
                    print(f"   🔐 No autorizado")
                else:
                    print(f"   ⚠️  Otro error: {response.status_code}")
                    
            except requests.exceptions.Timeout:
                print(f"   ⏰ Timeout")
            except Exception as e:
                print(f"   💥 Error: {str(e)[:50]}...")
    
    return None

print("🔧 También puedes ejecutar 'probar_endpoints_buk()' para encontrar el endpoint correcto")

🔧 Ejecuta 'probar_api_buk_simple()' para probar la API con un rango pequeño
🔧 También puedes ejecutar 'probar_endpoints_buk()' para encontrar el endpoint correcto


In [ ]:
# PROBAR DIFERENTES FORMATOS DE API KEY
# =====================================================================

def probar_formatos_api_key():
    """
    Prueba diferentes formatos de API Key para encontrar el correcto
    """
    print("🔑 PROBANDO DIFERENTES FORMATOS DE API KEY")
    print("=" * 50)
    
    api_key = 'JVZhehjdRWdmx2reD5mkKRHA'
    base_url = "https://demo.buk.cl/api/v1/chile/absences/absence"  # URL corregida
    
    # Diferentes formatos posibles de API Key
    formatos_headers = [
        {'X-API-Key': api_key},  # Formato principal usado en bloque 1
        {'Authorization': f'API-KEY {api_key}'},
        {'Authorization': f'ApiKey {api_key}'},
        {'Authorization': f'Bearer {api_key}'},
        {'Authorization': api_key},
        {'X-API-KEY': api_key},
        {'X-Api-Key': api_key},
        {'api-key': api_key},
        {'apikey': api_key},
        {'Authorization': f'Token {api_key}'}
    ]
    
    # Parámetros de prueba
    test_params = {
        'from': '2024-01-01T00:00:00',
        'to': '2024-01-31T23:59:59'
    }
    
    for i, auth_header in enumerate(formatos_headers, 1):
        print(f"\n🧪 Formato {i}: {auth_header}")
        
        # Agregar headers base
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            **auth_header  # Combinar con el header de autenticación
        }
        
        try:
            response = requests.get(base_url, headers=headers, params=test_params, timeout=15)
            
            print(f"   Status: {response.status_code}")
            
            if response.status_code == 200:
                print(f"   ✅ ¡FORMATO CORRECTO ENCONTRADO!")
                print(f"   📋 Header exitoso: {auth_header}")
                
                if response.content:
                    try:
                        data = response.json()
                        print(f"   📊 Datos recibidos: {type(data)}")
                        if isinstance(data, list) and data:
                            print(f"   📈 {len(data)} registros encontrados")
                        return auth_header, data
                    except:
                        print(f"   📄 Respuesta no-JSON pero exitosa")
                        return auth_header, response.text[:200]
                        
            elif response.status_code == 401:
                print(f"   ❌ No autorizado - formato incorrecto")
            elif response.status_code == 403:
                print(f"   ❌ Acceso denegado")
            elif response.status_code == 404:
                print(f"   ❌ Endpoint no encontrado")
            else:
                print(f"   ⚠️  Error {response.status_code}: {response.text[:100]}...")
                
        except requests.exceptions.Timeout:
            print(f"   ⏰ Timeout")
        except Exception as e:
            print(f"   💥 Error: {str(e)[:50]}...")
    
    print("\n❌ Ningún formato de API Key funcionó")
    return None, None

# FUNCIÓN DE PRUEBA COMPLETA
def validar_configuracion_completa():
    """
    Valida toda la configuración: URL, endpoint, API Key y parámetros
    """
    print("🔍 VALIDACIÓN COMPLETA DE CONFIGURACIÓN BUK")
    print("=" * 55)
    
    # Probar formatos de API Key
    formato_correcto, datos = probar_formatos_api_key()
    
    if formato_correcto:
        print(f"\n✅ CONFIGURACIÓN ENCONTRADA:")
        print(f"   🔗 URL: https://cramer.buk.cl/api/v1/chile/absences/absence")
        print(f"   🔑 Header: {formato_correcto}")
        print(f"   📊 Datos: {'Disponibles' if datos else 'No disponibles'}")
        
        # Actualizar la configuración global
        global headers
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'User-Agent': 'BukDataExtractor/1.0',
            **formato_correcto
        }
        
        print(f"\n🔄 Headers globales actualizados automáticamente")
        return True
    else:
        print(f"\n❌ No se pudo encontrar una configuración válida")
        print(f"💡 Sugerencias:")
        print(f"   1. Verifica que tu API Key sea correcta")
        print(f"   2. Confirma que tienes acceso al endpoint")
        print(f"   3. Revisa la documentación de BUK para el formato correcto")
        return False

print("🎯 FUNCIONES DE VALIDACIÓN DISPONIBLES:")
print("   - probar_formatos_api_key(): Prueba diferentes formatos de API Key")
print("   - validar_configuracion_completa(): Validación completa y actualización automática")
print("\n💡 Ejecuta 'validar_configuracion_completa()' para encontrar y configurar automáticamente")

🎯 FUNCIONES DE VALIDACIÓN DISPONIBLES:
   - probar_formatos_api_key(): Prueba diferentes formatos de API Key
   - validar_configuracion_completa(): Validación completa y actualización automática

💡 Ejecuta 'validar_configuracion_completa()' para encontrar y configurar automáticamente
